In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('./yonhapall.pkl')

In [3]:
len(df)

972293

In [4]:
df.head(5)

,article,date,section,title,article_len
0,7일 서울 목동야구장에서 열린 프로야구 한국시리즈 3차전 넥센 히...,20141107220335,스포츠,역점 2점홈런 날리는 박한이,102
1,7일 서울 목동야구장에서 열린 프로야구 한국시리즈 3차전 넥센 히...,20141107220450,스포츠,역점 2점홈런 날리는 박한이,102
2,7일 서울 목동야구장에서 열린 프로야구 한국시리즈 3차전 넥센 히...,20141107220417,스포츠,역점 2점홈런 날리는 박한이,102
3,7일 서울 목동야구장에서 열린 프로야구 한국시리즈 3차전 넥센 히...,20141107220436,스포츠,역전의 사나이 박한이,107
4,7일 서울 목동야구장에서 열린 프로야구 한국시리즈 3차전 넥센 히...,20141107220342,스포츠,환호하는 박한이,103


## 섹션 분리

In [5]:
df['sec']=df['section'].apply(lambda x: x.replace('일반', ''))

In [6]:
counts=df.groupby('sec').size()
sorted_counts=counts.sort_values(ascending=False)
sorted_counts

sec
지방행정        115767
사회          104446
국제           87786
정치           83125
경제           82878
야구           24383
기업           19192
산업           17852
문화           15581
중앙행정         14975
축구           14301
스포츠          13920
연예           13540
외교           13004
재테크          12001
국방           11565
대기업          10699
북한            9529
경찰            9257
교육            9056
IT            9052
골프            8023
국회            7492
소비자           6874
농구            5697
편집            5541
금융·증권         5455
대학교육          5207
사건            4778
증권시장          4741
             ...  
임대업              4
신용카드사            3
장애인올림픽           3
수상경기             3
익스트림게임           3
금시세              2
기업분석             2
우슈               2
교포               2
무속신앙             2
국내대회             2
헤드라인             1
개발원조             1
OANA             1
아동청소년복지          1
유럽연합             1
비치발리볼            1
세팍타크로            1
장애아시안게임          1
기타국내종합경기         1
전국소년체전           1
북한사회    

In [7]:
sorted_counts.to_csv('./yonhapsec.csv')

In [8]:
len(df)

972293

In [9]:
def high_sec(section):
    if section in ('사회', '사건', '사고', '검찰', '법원', '해상사고', '육상사고', '사기', '폭행', '자살') :
        return '사회'
    elif section in ('정치', '북한', '남북교류', '남북회담','청와대') :
        return '정치'
    elif section in ('경제', '증권시장', '금융·증권', '부동산', '해외증시', '자본시장', '경제정책', '금융시장', '외환시장', '수출', 'CEO', '외국계기업', '인수합병', '물가', '유가', '기업실적발표', '부도파산', '기업공시', '채권', '펀드', '금리') :
        return '경제'
    elif section in ('문화', '영화', '전시공연', '문화시설', '축제', '문화정책', '연극', '애니매이션', '무형문화재', '클래식', '무용') :
        return '문화'
    elif section in ('기상', '의학', '날씨', '생활정보', '보건', '생활건강', '식물', '요리·맛집', '건강·웰빙') :
        return '생활'
    elif section in ('과학기술', '항공우주산업', '첨단범죄', '자연과학', '생명공학') :
        return '과학'
    elif section in ('국제', '국제기구') :
        return '국제'
    else :
        return ''
    
        

In [10]:
df['sec1']=df['sec'].apply(high_sec)



In [11]:
sec1_counts=df.groupby('sec1').size()
sec1_sorted_counts=sec1_counts.sort_values(ascending=False)
sec1_sorted_counts

sec1
      519371
사회    118875
경제    112307
정치     98303
국제     87837
문화     24357
생활      8615
과학      2628
dtype: int64

In [12]:
df_cate = df[df['sec1']!='']

In [13]:
df_cate=df_cate.reset_index(drop=True)
len(df_cate)

452922

In [14]:
counts=df_cate.groupby('sec1').size()
section_idx=enumerate(counts.keys())
section_to_int = {word: ii for ii, word in section_idx}
section_to_int

{'경제': 0, '과학': 1, '국제': 2, '문화': 3, '사회': 4, '생활': 5, '정치': 6}

In [15]:
def set_section_idx(section):
    return section_to_int[section]

In [16]:
df_cate['sec_idx']=df_cate['sec1'].apply(set_section_idx)
docs=df_cate['article']

In [17]:
df_cate.drop(['date', 'section','sec'], axis = 1, inplace = True)

In [18]:
df_cate.to_pickle('yonhap_ready.pkl')

In [211]:
%%time
from konlpy.tag import Mecab; m = Mecab()
#pos = lambda d: ['/'.join(p) for p in m.pos(d)]
def noun_tagger(text):
    #return [pos[0] for pos in t.pos(text) if pos[1] in ['SL','SH','SN','NN','NNG', 'NNP']]
    #return [pos[0] for pos in m.pos(text) if (pos[1] in ['SL','SH','SN']) or pos[1].startswith('NN')]
    return [pos[0] for pos in m.pos(text) if pos[1].startswith('NN')]


CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.42 ms


In [212]:
stopwords_news=['지난해', '중','말', '뒤', '곳', '군', '위', '개', '간', '건', '이날', '도', '등', '명', '시', '앞', '원', '분', '회', '년', '것', '씨', '일', '월','오전','오후']

In [213]:
%%time
import numpy as np;
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(tokenizer=noun_tagger,stop_words=stopwords_news, ngram_range=(1,3), min_df=2, max_df=.25)
vect_weights = vect.fit_transform(docs)
#sorted(vect.vocabulary_, key=vect.vocabulary_.get, reverse=True)
weights = np.asarray(vect_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': vect.get_feature_names(), 'weight': weights})

CPU times: user 26min 27s, sys: 25.8 s, total: 26min 52s
Wall time: 26min 53s


In [214]:
weights_df.sort_values(by='weight', ascending=False).head(50)

,term,weight
2722259,대통령,0.008269
9440335,중국,0.008066
3664119,미국,0.006787
7674869,의원,0.005966
4573158,북한,0.005923
2764477,대표,0.005839
8887803,정부,0.005522
782675,경찰,0.005497
6120161,시장,0.005140
11470229,한국,0.004989
